# 任务 2：使用 SageMaker 特征存储

## 任务 2.1：特征存储选项

主要可通过三种方式在 Amazon SageMaker 中存储特征：
1.在完成预处理步骤并添加特征后，使用 Amazon SageMaker 特征存储作为 Amazon SageMaker Data Wrangler 目的地。
2.从 SageMaker Data Wrangler 导出一个支持提供特征定义、创建特征组并将数据摄取到 SageMaker 特征存储中的笔记本。
3.在支持提供特征定义、创建特征组并将数据摄取到 SageMaker 特征存储中的自定义笔记本中使用 SageMaker Python SDK。

以下各节将介绍上述三个选项。

### 使用 SageMaker 特征存储作为 SageMaker Data Wrangler 目的地

您可以在 Amazon SageMaker Studio 中使用 **Add destination**（添加目的地）选项将 SageMaker 特征存储添加为目的地。在 SageMaker Data Wrangler 中完成预处理步骤后，您可以在流程中选择 **Add destination**（添加目的地）。SageMaker Studio 将指导您创建特征组，并完成必要的步骤，以将您的预处理数据摄取到 SageMaker 特征存储中。

有关如何将 SageMaker 特征存储添加为目的地的更多信息，请参阅 [在不使用代码的情况下在 Amazon SageMaker 中轻松创建和存储特征](https://aws.amazon.com/blogs/machine-learning/easily-create-and-store-features-in-amazon-sagemaker-without-code/#save_features_to_feature_store)。

有关如何在 SageMaker Studio 中创建特征组的更多信息，请参阅 [将 Amazon SageMaker 特征存储与 Amazon SageMaker Studio 结合使用](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html)。

### 从 SageMaker Data Wrangler 导出笔记本

您可以使用 SageMaker Studio 中的 **Export to**（导出到）选项来创建特征组。在使用 **Export to**（导出到）选项时，您可以创建一个笔记本，该笔记本应包含创建特征组所需的所有命令。

进行一些自定义设置后，您可以运行该笔记本以执行以下操作：
- 根据您的数据集创建特征定义。
- 使用特征定义创建一个特征组。
- 将特征组存储在 SageMaker 特征存储中。
- 设置处理作业的输入和输出。

本实验类似于导出的笔记本。本实验侧重于导出笔记本的第一部分。它将展示如何将数据摄取到特征组中，以及如何从在线存储和离线存储中提取记录。

### 在自定义笔记本中使用 SageMaker Python SDK

在本实验中，您将创建一个特征组并从在线存储和离线存储中提取记录。您将使用自定义笔记本来了解 SageMaker 特征存储如何运作。您将在任务 2 中设置环境。接着，您将完成以下任务：

任务 2.3：设置 SageMaker 特征存储
- 在笔记本文件中创建特征。
- 在 SageMaker 特征存储中创建一个特征组。
- 确认已创建特征组。
- 在 SageMaker Studio 中查看该特征组。

任务 2.4：查询在线和离线存储
- 将数据摄取到特征组。
- 从在线存储中提取记录。
- 使用 Amazon Athena 从离线存储中提取记录。

## 任务 2.2：环境设置

安装软件包和依赖项。

In [ ]:
#install-dependencies

import boto3
import json
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time
import uuid
import random
from sagemaker.session import Session
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup


region = boto3.Session().region_name
sess = boto3.Session(region_name=region)
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

导入经过处理的客户数据集。

In [ ]:
#explore-dataset

column_list = ['income','age','education','education_num','capital_gain','capital_loss','hours_per_week','sex','workclass','marital_status','occupation','relationship','race']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

然后，在表中查看数据集示例。

In [ ]:
#view-dataset

lab_test_data.dtypes
lab_test_data.head()

## 任务 2.3：设置 SageMaker 特征存储

创建特征以帮助训练您的模型。从您的 Amazon SageMaker Data Wrangler 实验获取经过处理的数据，以使用 SageMaker 特征存储创建一个特征组。

- 在笔记本文件中创建特征。
- 在 SageMaker 特征存储中创建一个特征组。
- 确认已创建该特征组。
- 在 SageMaker Studio 中查看该特征组。

### 任务 2.3.1：在笔记本文件中创建特征

要创建特征组，您需要为 **record_identifier_name** 和 **event_time_feature_name** 分配列。要满足此要求，请将 **record** 和 **event_time** 列添加到数据集。
- **record_identifier_name** 是指在特征组的特征定义中定义的特征名称之一。在本实验中，您将创建一个名为 **record** 的唯一 ID 列。
- **event_time_feature_name** 是新事件发生的时间点，对应特征组中记录的创建或更新。特征组中的所有记录都必须具有相应的事件时间。该时间可用于跟踪记录随着时间的推移而发生的变化。在本实验中，您需要创建一个名为 **event_time** 的列。

有关记录标识符名称或事件时间的更多信息，请参阅 [开始使用 Amazon SageMaker 特征存储](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-concepts.html)。

In [ ]:
#add-required-columns

# Add record and event_time columns
current_time_sec = int(round(time.time()))
lab_test_data.insert(0, 'record', range(0, 0 + len(lab_test_data)))
lab_test_data.insert(1, 'event_time', [current_time_sec]*len(lab_test_data))
lab_test_data['record'] = lab_test_data['record'].astype('string')
lab_test_data['event_time'] = lab_test_data['event_time'].astype('float64')

# Set the record-and-event_time-feature-names
record_identifier_feature_name = 'record'
event_time_feature_name = 'event_time'

# View the dataset
print(lab_test_data.dtypes)
lab_test_data.head()

在数据处理过程中，您可以根据需求随时使用 SageMaker Python SDK 或 SageMaker Studio 添加新的特征。

在本实验中，作为数据预处理步骤的一部分，请使用 SageMaker Python SDK 添加一个特征。该特征是数据集中两列的加权组合，将有助于更高效地训练模型。

创建一个 workability 特征，结合 age 和 hours_per_week 列来识别在职业生涯中更进一步的客户。

In [ ]:
#add-feature

lab_test_data = lab_test_data.assign(
    workability = 0.5*lab_test_data.age + 0.5*lab_test_data.hours_per_week)

lab_test_data.head()

### 任务 2.3.2：在 SageMaker 特征存储中创建一个特征组

要将特征摄取到 SageMaker 特征存储中，首先要定义属于该特征组的所有特征的特征定义（特征名称和数据类型）。

一个特征对应数据集中的一列。特征组是特征集合的预定义架构。特征组中的每个特征都有指定的数据类型和名称。特征组中的一项记录对应数据帧中的一行。特征存储是特征组的集合。

有关 SageMaker 特征存储的更多信息，请参阅 [使用 Amazon SageMaker 特征存储创建、存储和共享特征](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html)。

要开始创建特征定义的流程，请列出每个特征的架构。

In [ ]:
#list-column-schemas

column_schemas = [
    {
        "name": "record",
        "type": "string"
    },
    {
        "name": "event_time",
        "type": "float"
    },
    {
        "name": "income",
        "type": "string"
    },
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "education",
        "type": "float"
    },
    {
        "name": "education_num",
        "type": "float"
    },
    {
        "name": "capital_gain",
        "type": "long"
    },
    {
        "name": "capital_loss",
        "type": "long"
    },
    {
        "name": "hours_per_week",
        "type": "long"
    },
    {
        "name": "sex",
        "type": "float"
    },
    {
        "name": "workclass",
        "type": "array"
    },
    {
        "name": "marital_status",
        "type": "array"
    },
    {
        "name": "occupation",
        "type": "array"
    },
    {
        "name": "relationship",
        "type": "array"
    },
    {
        "name": "race",
        "type": "array"
    },
    {
        "name": "workability",
        "type": "float"
    }
]


现在，您已经定义了架构，请为特征定义创建输入。为 FRACTIONAL 和 INTEGRAL 的 float 和 long 数据集值设置一个类型映射。

然后，创建您的特征定义，设置所定义的架构中所有列的 **feature_name** 和 **feature_type** 值。

In [ ]:
#create-feature-definitions

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

在 SageMaker 特征存储中，特征组可以是在线的，也可以是离线的，或者两者兼而有之。在本实验中，您将使用在线存储和离线存储，因此您需要将 **enable_online_store** 设置为 **True**。
- 在线存储主要用于支持需要低毫秒级延迟读取和高吞吐量写入的实时预测。
- 离线存储主要用于批量预测和模型训练。离线存储是一种仅可附加的存储，可用于存储和访问历史特征数据。

如果您的特征存储同时设置为在线存储和离线存储，那么所有摄取到在线存储的特征都会被复制到离线存储中。

配置特征组，指定以下设置的选项：
- **feature_group_name**：特征组的名称
- **feature_store_offline_s3_uri**：SageMaker 特征存储在特征组离线存储中写入数据的 Amazon Simple Storage Service (Amazon S3) 存储桶位置。
- **enable_online_store**：控制是否启用在线存储

In [ ]:
#configure-feature-store

# flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = "DataWranglerLab"
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

feature_store_offline_s3_uri = 's3://' + bucket
enable_online_store = True

现在，您的特征存储已配置完毕，请设置 AWS 区域并启动会话。然后，设置 SageMaker 客户端和特征存储运行时。最后，设置特征存储会话。

在设置特征存储会话时，就已定义了 **boto_session**、**sagemaker_client** 和 **sagemaker_featurestore_runtime_client** 值。

In [ ]:
#set-up-sagemaker-feature-store-session

sagemaker_client = sess.client(service_name='sagemaker', region_name=region)
featurestore_runtime = sess.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=sess,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

使用之前配置的参数初始化特征组，并调用特征存储 API 来创建特征组。

In [ ]:
#initialize-feature-group

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=enable_online_store
)

### 任务 2.3.3：确认已创建特征组

现在，您的特征组应已准备就绪。确认已成功创建特征组。

耐心等待，直到使用 Describe API 确定特征组已准备就绪。此函数会检查 Describe API 返回的响应并等待状态显示为 **Created**（已创建）。

In [ ]:
#wait-for-describe

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

使用 ListFeatureGroups API 列出这些特征组。

In [ ]:
#list-feature-groups

response = sagemaker_client.list_feature_groups()
print(json.dumps(response, indent=4, sort_keys=True, default=str))

### 任务 2.3.4：在 SageMaker Studio 中查看特征组

您已使用 SageMaker Python SDK 创建一个 SageMaker 特征组。现在，请在 SageMaker Studio 中查看该特征组以了解其他详细信息。

1.在笔记本左侧的导航选项卡中，选择 **SageMaker home**（SageMaker 主页）图标。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_5.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_5.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览特征组时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完特征组后，通过选择 **lab_5.ipynb** 选项卡返回至笔记本。

2.展开 **Data**（数据）部分，然后选择 **Feature Store**（特征存储）。
3.您刚刚创建的特征组会出现在 <b>Feature Store</b>（特征存储）选项卡中。您可以查看关于该特征组的详细信息。要查找更多详细信息，请选择以 **FG-DataWranglerLab-** 开头的特征组。在 SageMaker Studio 中浏览 SageMaker 特征存储时，请查看以下详细信息：
    - **Feature**（特征）：描述特征组中的所有特征，包括 **Type**（类型）以及基于 **event_time** 列的特征创建时间。
    - **Details**（详细信息）：概述特征组的元数据，包括**特征组状态**、您之前在笔记本中设置的**记录标识符**、设置为在线/离线的**存储类型**以及可用于通过 Athena 从离线特征存储中查询数据的**表名**。
    - **Sample query**（示例查询）：提供可用于从离线特征存储中查询数据的多个示例查询。

## 任务 2.4：查询在线和离线存储

您已创建了一个特征组。现在，您需要将数据摄取到该特征组、从在线存储中提取记录，并使用 Athena 从离线存储中提取记录。

- 将数据摄取到特征组。
- 从在线存储中提取记录。
- 使用 Athena 从离线存储中提取记录。

### 任务 2.4.1：将数据摄取到特征组

创建特征组后，您需要将数据放入其中。通过 SageMaker Python SDK 执行使用 **ingest()** 的 PutRecord API 调用。每当您首次创建特征组或想要添加更多记录时，您都可以将记录摄取到特征组中。

对于此数据集，摄取需要 3–5 分钟才能完成。此单元格完成后，将显示如下所示的输出：

**IngestionManagerPandas(feature_group_name='FG-DataWranglerLab-13ee4f26', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fdb7fccee60>, sagemaker_session=<sagemaker.session.Session object at 0x7fdb82a900d0>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])**

In [ ]:
#ingest-records

feature_group.ingest(data_frame=lab_test_data, wait=True)

### 任务 2.4.2：从在线存储中提取记录

在线存储在处理推理任务时尤为有用，因为您可以快速返回特征子集。

现在，您的数据已被摄取，您需要使用 **get_record** 从在线存储中提取记录。

In [ ]:
#get-record

record = random.randint(0, len(lab_test_data.index)-1)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=str(record))

print(json.dumps(sample_record, indent=4, sort_keys=True, default=str))

现在，请使用 **batch_get_record** 从特征组中获取几项记录。系统已经为您选择了几项记录，但您可以随时更改 **records_list** 中列出的记录。

In [ ]:
#batch-get-record

records_list = ["7789", "5646", "309", "24528"]

batch_records = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": feature_group_name,
            "RecordIdentifiersValueAsString": records_list,
        }
    ]
)

print(json.dumps(batch_records, indent=4, sort_keys=True, default=str))

### 任务 2.4.3：使用 Athena 从离线存储中提取记录

现在，您已经从在线存储中提取了记录，请使用 Athena 从离线存储中提取记录。

您可以在训练和优化模型时查询完整的数据集，也可以查询记录的子集以进行推理。由于 SageMaker 特征存储会为每项记录保留一个事件时间，因此您可以使用过去特定时间的确切特征集来训练模型，而无需面临包含该时间以外数据造成的风险。

您将如何调整查询以更改从离线存储中返回的数据子集？

首先，选择您的查询设置。您可以自定义查询，以查看存储在特征组中的数据的任何子集。以下查询是基本的 SELECT 查询定义。

In [ ]:
#query-settings
# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup='primary',
        ConfigurationUpdates={
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

#Create the query
query = feature_group.athena_query()
table = query.table_name
query_string = f'SELECT * FROM "{table}" '
output_location = f's3://{bucket}/query_results/'

print(f'Athena query output location: \n{output_location}')

设置选项后，运行查询，并以表格形式显示结果。

In [ ]:
#run-athena-query

query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

### 总结

恭喜！ 您已经使用 SageMaker 特征存储为 SageMaker Studio 和 SageMaker Python SDK 中的特征组创建了特征定义。有了新创建的特征组，您就可以在下一个实验中使用 SageMaker Experiments 训练和优化您的模型了。甚至在课程的后面部分，由于使用在线存储可实现低延迟的 GetRecord 功能，SageMaker 特征存储在为推理请求补充数据方面也会很有用。您将在下一个实验中继续使用此客户收入数据集。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。